In [7]:
# helper function for input reading

import platform
import sys
# Plattform erkennen
IS_WINDOWS = platform.system() == "Windows"

if IS_WINDOWS:
    import msvcrt

    def get_key():
        """Liest eine Taste unter Windows (ohne Enter)."""
        ch = msvcrt.getch()
        try:
            return ch.decode("utf-8").lower()
        except UnicodeDecodeError:
            return ""
else:
    import tty
    import termios

    def get_key():
        """Liest eine Taste unter Linux/Mac (ohne Enter)."""
        fd = sys.stdin.fileno()
        old_settings = termios.tcgetattr(fd)
        try:
            tty.setraw(fd)
            ch = sys.stdin.read(1)
        finally:
            termios.tcsetattr(fd, termios.TCSADRAIN, old_settings)
        return ch.lower()

In [8]:
# automatically generate stretch segment entries for stretches that have no stretch_segment entries yet

import pandas as pd
import keyboard
import numpy as np

STRETCHES_PATH = './stretches.csv'
STRETCH_SEGMENTS_PATH = './stretch_segments.csv'

stretches = pd.read_csv(STRETCHES_PATH, dtype=str).dropna(how='all')

print("Hinweis: Ein Stretch beinhaltet die Knoten, zwischen denen der Stretch verläuft. Die minimale Anzahl ist daher 1")


def optionally_add_stretch_segments(row):
    stretch_segments = pd.read_csv(STRETCH_SEGMENTS_PATH, dtype=str).dropna(how='all')
    
    stretch_id = row['stretch_id']
    stretch_name = row['stretch_name']
    
    # skip if stretch already has segments
    if len(stretch_segments[stretch_segments['stretch_id'] == stretch_id]) > 0:
        return
    
    # ask how many segments
    no_segments = int(input(f'Wie viele Segmente hat {stretch_name}?'))
    
    bounds = []
    
    if no_segments == 1:
        bounds = [(0,100)]
        
    elif no_segments == 2:
        bounds = [(0,50), (50,100)]
        
    elif no_segments > 2:
    # split progress percentages over segments. treating the nodes as being in the middle of a segment and therefore receiving only half of the progress.
        end_segment_size = (100 / (no_segments - 1)) / 2
        middle_segment_size = 100 / (no_segments - 1)
        
        no_middle_segments = no_segments - 2

        bounds.append((0,end_segment_size))
        for i in range(no_middle_segments):
            lower_bound = end_segment_size + i * middle_segment_size
            upper_bound = end_segment_size + (i + 1) * middle_segment_size
            bounds.append((lower_bound, upper_bound))
        bounds.append((100 - end_segment_size, 100))    
        
        bounds = [(int(x), int(y)) for x,y in bounds]

    for i, bounds in enumerate(bounds):
        stretch_segments = pd.concat([stretch_segments, pd.DataFrame(data={'stretch_id':[stretch_id],'stretch_name':[stretch_name],'segment_number':[i],'lower_bound':[bounds[0]],'upper_bound': [bounds[1]]})])
        stretch_segments.to_csv(STRETCH_SEGMENTS_PATH, index=False)
        
    
    
    
stretches.apply(optionally_add_stretch_segments, axis=1)
    


Hinweis: Ein Stretch beinhaltet die Knoten, zwischen denen der Stretch verläuft. Die minimale Anzahl ist daher 1


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
dtype: object